# A Step-by-Step tutorial for usage of ISMGCC

## Table of content

1. Prepare the environment
2. Decompose the input data cube with gausspy+
3. Use ISMGCC to find structures
4. Run a parameter grid of ISMGCC

## Prepare the environment 

```bash
conda create -n ismgcc-demo python=3.9
conda activate ismgcc-demo
python -m pip install astropy lmfit networkx numpy scipy tqdm matplotlib 
wget https://github.com/Haoran-Feng/gausspyplus/archive/refs/heads/master.zip
```

